# Module 3: Unsupervised Learning

Welcome to the third module of this series! In this module you will first have a deeper look at what unsupervised learning is and where it can be used. You will then explore one of the main use cases of unsupervised learning, namely dimensionality reduction.

**Module Overview**
1. [Introduction to Unsupervised Learning](#what-is-unsupervised)
2. [Dimensionality Reduction](#dim-red)<br>
    2.1 [Main Usage](#why-used)<br>
    2.2 [PCA](#pca)<br>
    2.3 [t-SNE](#tsne)<br>
    2.4 [UMAP](#umap)
3. [Conclusion](#conclusion)

**Dataset**

In this module we will work with the already preprocessed Swiss Food Composition dataset from Module 1. You can find the preprocessed dataset in `data/swiss_food_composition_proc.csv`. 

As a quick recap, this is the resulting dataset after:
- removing the samples and features with more than 20% of missing values,
- splitting the dataset in train and test sets,
- imputing missing values,
- standardizing the remaining data.

Note that in this module, we will not need the train and test splits since in the unsupervised learning case we do not use any target variables.

<a id='what-is-unsupervised'></a>
## 1. Introduction to Unsupervised Learning

Unsupervised machine learning is a type of machine learning that uses algorithms to analyze and cluster unlabelled datasets. Thus the used data either **does not have any labels that we can learn** and later predict or we can decide to **ignore the labels**. In unsupervised learning the models learn patterns in the data. Some use cases of such models include dimensionality reduction, clustering, data generation and anomaly detection: 

* In the case of [dimensionality reduction](#dim-red), we compress a dataset with many features to a dataset with fewer features. 
* When we do unsupervised learning with the goal of [clustering](https://scikit-learn.org/stable/modules/clustering.html), we try to find groups within the dataset so that we can group similar samples together. 
* In the case of [data generation](https://towardsdatascience.com/5-best-python-synthetic-data-generators-and-how-to-use-them-when-you-lack-data-f62bcf62d43c), we use the unlabelled dataset to learn its structure such that we can generate similar synthetic samples. 
* For [anomaly detection](https://towardsdatascience.com/5-anomaly-detection-algorithms-every-data-scientist-should-know-b36c3605ea16), we employ unsupervised models to identify outliers in the dataset. Outliers are samples that do not resemble the majority of the dataset's samples. 

[Fig. 1](#unsup_learn) illustrates the machine learning pipeline for unsupervised learning. As in supervised learning, the trained unsupervised model generates an output. The output corresponds to what the model has learned from the data. In the case of dimensionality reduction, the output will correspond to the dataset with fewer features. As you can see in the figure, the number of features `n'` in the new dataset is smaller than the original number of features `n`. 

One thing to note is the absence of the train-test split step. In supervised learning, we split the data into train and test such that we can assess how well the model trained on the train set performs on the "unseen" data in the test set. By evaluating the model's performance on the test set, we can quantify how well it generalizes to new, unseen data. In unsupervised learning the primary objective is to uncover hidden structures in the data, rather than making accurate predictions based on provided labels. Since there's no ground truth to compare against, the notion of "testing" in the same sense as supervised learning is not always applicable. Hence, there is usually no need for a train-test split step in unsupervised learning. There are specific cases where a an unsupervised model can be evaluated indirectly and we would perform a train-test split. For example, in the case of dimensionality reduction, we can compare its impact on supervised learning tasks by comparing the performance of supervised learning models on the original and reduced datasets (you will see this in our mini-project).



<center>
    <a id="unsup_learn"></a>
    <img src="images/part3_unsupervised/unsupervised_learning_dimred.jpg" alt="ML Unsupervised Learning" width="90%">
    <center><figcaption><em>Figure 1: Unsupervised Learning</em></figcaption></center>
</center>


💡 **CHECKPOINT:**
- How does supervised learning differ from unsupervised learning? Think of some example use cases for each.

<a id='dim-red'></a>
## 2. Dimensionality Reduction

<a id='why-used'></a>
###  2.1 Main Usage

High-dimensional datasets are datasets that have a lot of features compared to their number of samples. **Dimensionality reduction** is used to **summarize the information content of a high-dimensional dataset** by transforming it onto a new feature space of **lower dimensionality**. The dimensionality reduction algorithms identify the most important features and transform the data in a way that preserves most of the information. They also offer many benefits, particularly in terms of making high-dimensional results easier to visualize and explain to audiences, removing noise from the dataset and speeding up model training.

The more features a datasets has, the more samples are needed so that the ML models can learn it. This also known as **the curse of dimensionality**. Reducing the dimensionality of the dataset can enhance the performance of the trained ML model because it reduces the effects of the curse of dimensionality.  Therefore, one of the main goals of dimensionality reduction is to **reduce the number of features in the dataset while retaining the most important information**.

All dimensionality reduction techniques belong to the group of unsupervised learning algorithms. Some of these techniques include: PCA (Principal Component Analysis), t-SNE (t-distributed stochastic embedding) and UMAP (Uniform Manifold Approximation and Projection). Below we will explore **PCA**, **t-SNE** and **UMAP** on the *processed Swiss food composition dataset*. We will start by importing the necessary libraries and reading the dataset.

In [1]:
import os
import warnings
# we choose to ignore this depracation warning caused by umap
warnings.filterwarnings("ignore")
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from umap import  UMAP
import pandas as pd

# we need these packages for visualizing the results
import seaborn as sns
import ipywidgets as widgets
from matplotlib import pyplot as plt
import numpy as np
from IPython.display import display, clear_output

In [2]:
# we will work with the preprocessed dataset
filepath = os.path.join("data", "swiss_food_composition_proc.csv")
dataset = pd.read_csv(filepath, index_col='ID')
# save the numerical columns in the cols variable
numerical_cols = dataset.select_dtypes(include='number').columns
dataset.head()

,name,category,energy_kcal,fat_g,fatty_acids_sat_g,fatty_acids_monounsat_g,fatty_acids_polyunsat_g,cholesterol_mg,carbohydrates_g,sugars_g,...,potassium_mg,sodium_mg,chloride_mg,calcium_mg,magnesium_mg,phosphorus_mg,iron_mg,iodide_µg,zinc_mg,split
ID,,,,,,,,,,,,,,,,,,,,,
2,Almond,fruits,2.383884,2.367211,0.007568,3.747869,1.491113,-0.536758,-0.392082,0.004006,...,1.588591,-0.159777,-0.148160,1.295299,4.040213,2.312243,0.749351,-0.106408,1.315695,train
3,"Almond, dry roasted, salted",nuts,2.458504,2.390231,0.007568,3.980279,1.745907,-0.536758,-0.285661,-0.117252,...,1.488645,-0.083438,0.108003,1.295299,4.816868,2.059374,0.906205,-0.088162,1.315695,train
4,"Almond, roasted, salted",nuts,2.527384,2.545613,0.020843,4.212688,1.825530,-0.536758,-0.419844,-0.138650,...,1.355384,-0.050088,0.108003,1.098111,4.622704,2.059374,0.749351,-0.088162,1.185016,test
5,"Amaranth, seed, cooked (without addition of fa...",cereals,-0.514820,-0.510234,-0.483593,-0.476516,-0.196900,-0.536758,0.084497,-0.431096,...,-0.377014,-0.159511,-0.149051,-0.098161,1.011260,0.226077,0.631711,-0.101432,-0.056440,test
6,"Amaranth, seed, raw",cereals,0.960362,-0.228244,-0.337572,-0.312462,0.121593,-0.536758,1.875138,-0.345502,...,0.722392,-0.158810,-0.132568,0.572278,4.234377,2.628328,2.984515,-0.087333,1.577054,train


Here we define two helper functions that will visualize the data after applying the dimensionality reduction techniques. Note that we select only the first two dimensions of the reduced dataset (which are all the features the dataset will have after applying dimensionality reduction). 

In [3]:
def create_scatter(X_1: np.array, X_2: np.array, y: np.array,
    selected_category: str, title: str):
    
    '''
    This function will create the scatter plot of points in low dimensions.
    It will display only the samples that belong to the `selected_category`.

    Args:
     - X_1: the first dimension of the data
     - X_2: the second dimension of the data
     - y: the category of the sample
     - selected_category: the category selected by the user
     - title: the title of the plot

     Returns: nothing
    '''

    color_mapping = {cat: i for i, cat in enumerate(np.unique(y))}
    
    _, ax = plt.subplots(figsize=(8, 6))

    for category in np.unique(y):
        category_mask = (y == category)
        
        if selected_category == "All categories":
            ax.scatter(X_1[category_mask], X_2[category_mask], \
                c='C{}'.format(color_mapping[category]), label='{}'.format(category), s=8)
        elif category == selected_category:
            ax.scatter(X_1[category_mask], X_2[category_mask], \
                c='C{}'.format(color_mapping[category]), label='{}'.format(category), s=8)
        else:
            ax.scatter(X_1[category_mask], X_2[category_mask], \
                c='gray', alpha=0.2, s=8)
    
    ax.set_xlabel('dim_1')
    ax.set_ylabel('dim_2')
    ax.set_title(title)
    ax.legend()
    sns.move_legend(ax, "center left", bbox_to_anchor=(1, 0.5))
    plt.show()


def visualize_data(data: pd.DataFrame, title: str):
    '''
    This method is used to initiate the process of data visualization. It splits
    the dimensionality reduced data in two arrays one for each of the dimensions
    and turns the categories in an array as well. Then it creates and displays a
    dropdown list, where the user can control the category displayed in the
    plot. The actual visualization happens in the create_scatter function.

    Args:
     - data: the dimensionality reduced data
     - title: the title of the plot that will be displayed

    Returns: nothing
    
    '''

    sns.set(
        rc={'figure.figsize':(7, 5), 'font.family': ['DejaVu Sans']},
        style='white'
    )

    X_1 = np.array(data[['dim_1']])
    X_2 = np.array(data[['dim_2']])
    y = np.array(data[['category']])

    category_dropdown = widgets.Dropdown(
        options=["All categories"] + list(np.unique(y)),
        value="All categories",
        description='Category:'
    )

    output = widgets.Output()

    def on_category_change(change):
        selected_category = change['new']
        with output:
            clear_output(wait=True)
            create_scatter(X_1=X_1, X_2=X_2, y=y, title=title, \
                selected_category=selected_category)

    category_dropdown.observe(on_category_change, names='value')

    display(category_dropdown, output)
    # this is for creating the initial plot
    on_category_change({'new': category_dropdown.value})  

<a id='pca'></a>
### 2.2 PCA

**Principal Component Analysis** is a **linear dimensionality reduction technique**. It projects the data points to the directions of the highest variance, since they contain the most important information in the dataset. The data is projected into a new subspace with less features than before. **The new axes of the data are orthogonal to each other and are the directions of the maximum variance in the dataset.**

[Fig. 2](#pca) gives an illustration of PCA:
<center>
    <a id="pca"></a>
    <img src="images/part3_unsupervised/PCA.jpg" alt="PCA" width="90%">
    <center><figcaption><em>Figure 2: PCA components</em></figcaption></center>
</center>

The left side of the figure shows the original data in two dimensions. We find the directions of highest variance and then project the data according to the subspace defined by these directions.
The red axes in the middle figure show the directions of highest variance. The data points are projected in these two directions, shown on the right hand side of the figure. In this example, there is no dimensionality reduction. The projected data points still have two dimensions after PCA, but they are in a new subspace defined by the directions of highest variance.

**NOTE**: In this figure, there is no dimensionality reduction, as both the original and the reduced datasets have two dimensions. Also, the reduced datasets do not necessarily always have 2 components. The number of components is a hyperparameter (see module 2 for the distinction between parameters and hyperparameters) set by the data analyst. Here we always choose 2 components to visualize the results and make the procedure more intuitive.

Another important concept related to dimensionality reduction is that of **variance explained**. In dimensionality reduction, variance explained stands for the amount of information retained when the original high-dimensional data is projected into a lower-dimensional space. In case of PCA, it orders the components in decreasing order of the amount of variance they explain. The first principal component explains the largest amount of variance in the data, the second component explains the second-largest amount of variance, and so on. By summing the variance of the first *n* principal components we choose to project the data on, we can quantify how much information is retained by reducing the dataset to *n* dimensions using PCA. Usually, the explained variance is reported as a percentage. For example, if the first three principal components of a dataset explain 60% of the total variance, it means that those three components capture 60% of the information present in the original data.

**PCA is sensitive to data scaling.** If variables in the dataset have different units of measurement, this can lead to biased results and inaccurate interpretations of the principal components. Therefore, it is important that the features are scaled before PCA is applied, especially if the features of the data are measured on different scales. Our dataset is already standardized from the first tutorial. So we are ready to apply PCA and visualize the results.

In [4]:
pca = PCA(n_components=2) # define the PCA object

# perform the PCA transformation and save the results in a dataframe
pca_components = pd.DataFrame(
    pca.fit_transform(dataset[numerical_cols]), # apply PCA here 
    columns=['dim_1', 'dim_2'], # define the new column names
    index=dataset.index
)

First, we define a PCA object that will perform the transformation. We specify the number of reduced dimensions, which in our case will be 2 (`n_components=2`). Then we apply the PCA transformation to the numerical part of our dataset by using the `fit_transform()` method of the `pca` object (`pca.fit_transform(dataset[numerical_cols])`). Note that this method takes the original dataset and reduces the 38 numerical features to 2 only. We save the new features in a dataframe. This dataframe will contain two columns `dim_1` and `dim_2`, one for each of the dimensions after PCA (`columns=['dim_1', 'dim_2']`). Finally, in order for the reduced dataset to have the same index as the original  dataset, we set the index to the dataset index (`index=dataset.index`).

In [5]:
# recreate the dataset but with the pca_components
data_after_pca = pd.concat([pca_components, dataset['category']], axis=1)
data_after_pca.columns = ['dim_1', 'dim_2', 'category'] # rename the columns
data_after_pca.head() # print the first 5 samples after PCA

,dim_1,dim_2,category
ID,,,
2,3.581816,-6.828132,fruits
3,4.037108,-7.033969,nuts
4,3.362831,-7.156479,nuts
5,-0.735546,0.174700,cereals
6,3.008247,-4.400588,cereals


To create the reduced dataset, we concatenate the `pca_components` with the *category* column of the dataset and form a new dataframe. The concatenation is horizontal (`axis=1`), meaning that the *category* is appended to  the right of the `pca_components` as a new column. We define the new columns of the dataframe to be `dim_1`, `dim_2` and `category`. You can see the first 5 samples in the output of the `head()` command. As you can see, the dataset now contains two numerical features and the category.

After this, we can visualize the data by using our helper functions defined before:

In [6]:
visualize_data(data_after_pca, 'PCA embeddings')

Dropdown(description='Category:', options=('All categories', 'alcoholic_beverages', 'cereals', 'dairy', 'fruit…

Output()

Here we can see where different food categories reside in the 2D space after applying PCA.

You can find more information regarding PCA in [Chapter 5 of the "Machine Learning with Pytorch and Scikit-Learn"](https://learning.oreilly.com/library/view/machine-learning-with/9781801819312/Text/Chapter_5.xhtml#_idParaDest-94) book.

💡 **CHECKPOINT:**
- Why do you think the samples from different categories are clustered together (categories are not well separated) after PCA?

<a id='t-sne'></a>
### 2.3 tSNE

**T-distributed Stochastic Neighbor Embedding (t-SNE)** is a **non-linear** dimensionality reduction technique. It is mainly used for visualizing high-dimensional datasets in low dimensions. t-SNE is useful in datasets where groups/clusters are not linearly separable. Usually, in these circumstances, linear dimensionality reduction techniques like PCA do not work well.
t-SNE projects data points into the low dimensional space by trying to preserve the distances they have in the high-dimensional space. Also, t-SNE requires the whole dataset to project it to the low-dimensional space and cannot be applied to new data points directly, without recalculating the models. You can consult [the original paper](https://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf) for more information regarding how t-SNE works. Alternatively, you can read [this blog post](https://towardsdatascience.com/t-sne-clearly-explained-d84c537f53a) for a more intuitive explanation.

Now we will proceed with the implementation of t-SNE. We will use the sklearn implementation of t-SNE to reduce the original dataset to 2 dimensions for visualizing it, just like we did with PCA in the previous section.

In order to  make t-SNE interactive, we define a function (`run_tsne`) that receives the **perplexity** as argument and uses t-SNE to reduce the dataset to 2 dimensions. Perplexity is one of the hyperparameters of t-SNE that controls the preservation of  local and global structure  of the data points. It determines the number of neighboring points that the algorithm checks for each point when determining the similarity between points in high-dimensional space. The larger the value of perplexity, the more neighbours will be considered for each data point. Large perplexity values favour preservation of global structure. On the other hand, low perplexity values favour preservation of local structure. Here we provide a slider to experiment with its different values and see the effects it has on the new dataset. You can check different values of the perplexity hyperparameter ranging from 10 to 100, but in practice typical values of perplexity are between 5 and 50. To learn more about the hyperparameters of t-SNE you can refer to [this article](https://distill.pub/2016/misread-tsne/).

Apart from this, the code that performs the dimensionality reduction is similar to that of PCA. First we define a `tsne` object that will perform the dimensionality reduction. It will reduce the original dataset to 2 dimensions. It is initialized with the following hyperparameters:
 - `n_components` - the number of features of the reduced dataset
 - `init='pca'` to initialize the t-SNE embedding, recommended in the original paper. 
 - `perplexity` - determined by the slider. 
 - `random_state=0` - to make the results deterministic
 
 After the `tsne` object is initialized, we use it to perform the dimensionality reduction. By using the `fit_transform()` method with the numerical features of the original dataset only, we get the reduced version with 2 features, named as `dim_1` and `dim_2`. We save the reduced dataset in a pandas dataframe called `tsne_components`. Then, we create a dataframe that will contain the two features of the dataset after applying t-SNE and the `category` column of the original dataset. In the end, we use our method `visualize_data` to visualize the new dataset after having reduced its dimensions using t-SNE.

In [7]:
def run_tsne(perplexity:int, dataset:pd.DataFrame, numerical_cols: list):
    '''
    This method is used to create the TSNE object that will perform the
    dimensionality reduction using the fit_transform method. The results of
    dimensionality reduction are saved in a pandas dataframe that contain the 2
    columns holding the new, reduced dimensions as well as a third column for
    the category of the sample. 

    Args:
     - perplexity: the perplexity parameter of TSNE, that will control the preservation
       of local structure.
     - dataset: a pandas dataframe containing the data
     - numerical_cols: a list of the columns containing numerical data

    Returns: nothing
    '''
    tsne = TSNE(n_components=2, init='pca', perplexity=perplexity, random_state=0)

    tsne_components = pd.DataFrame(
        data=tsne.fit_transform(dataset[numerical_cols]), 
        columns=['dim_1', 'dim_2'],
        index=dataset.index #this is important for the concatenation in the next command
        )
    
    data_after_tsne = pd.concat([tsne_components, dataset['category']], axis=1)
    data_after_tsne.columns = ['dim_1', 'dim_2', 'category']
    
    visualize_data(data_after_tsne, 
                   f'tSNE embeddings - perplexity={perplexity}')

In [8]:
perplexity_slider = widgets.IntSlider(value=30, min=10, max=100, step=5, description='Perplexity:')
widgets.interact_manual(run_tsne, perplexity=perplexity_slider, \
                        dataset = widgets.fixed(dataset), \
                        numerical_cols = widgets.fixed(numerical_cols));

interactive(children=(IntSlider(value=30, description='Perplexity:', min=10, step=5), Button(description='Run …

In [9]:
# You can use this cell to experiment more with the hyperparameters and compare 
# the plots. You will need to uncomment the below lines.

# perplexity_slider = widgets.IntSlider(value=30, min=10, max=100, step=5, description='Perplexity:')
# widgets.interact_manual(run_tsne, perplexity=perplexity_slider);

💡 **CHECKPOINT:**
- Experiment with different values of perplexity. What do you notice? How do different values of perplexity affect the grouping of different categories? At what values of perplexity do the samples of each category cluster together?

<a id='umap'></a>
### 2.4 UMAP

**Uniform Manifold Approximation and Projection (UMAP)** is another dimensionality reduction technique that we will explore. Just like t-SNE, it is **non-linear**, however, unlike t-SNE it tries to preserve both **the global structure of the dataset** and **the inter-cluster distances**. In addition, UMAP works better than t-SNE with large datasets. Just like t-SNE, UMAP also requires the whole dataset to project it to the low-dimensional space and cannot be applied to new data points directly, without recalculating the model. For more details on the theory of UMAP, you can read [this blog post](https://pair-code.github.io/understanding-umap/).

Now we will start with the implementation of UMAP. The code is similar to that of t-SNE, however here we work with two hyperparameters: `n_neighbors` and `min_dist`. `n_neighbors` is the most important hyperparameter of UMAP. It controls how UMAP balances the local and global structures. UMAP will put more effort in preserving local structure when the `n_neighbors` is low, since the number of points considered in high dimensions for each single point will be low (only the nearest neighbors). `min_dist` on the other hand, controls the minimum distance between points in the low-dimensional space. The lower the value, the closer the points are in low-dimensional space. We provide two sliders, one for the `n_neighbors` with values ranging from 10 to 100 and one for the `min_dist` with values ranging from 0 to 1. Note that in order for the visualization to appear, you must click the *Run Interact* button.


To reduce the dimensions of the original dataset, we use the `run_umap` function that takes as arguments the `n_neighbors` and the `min_dist` from the sliders. It will create the `umap` object, with the following hyperparameters:
 - `n_components=2` - the number of features of the reduced dataset
 - `min_dist` and `n_neighbors` - determined by the slider values
 - `random_state=0` - a seed to make results deterministic and reproducible for the same dataset and set of hyperparameters

After the `umap` object is initialized, we use it to perform the dimensionality reduction. By using the `fit_transform()` method with the numerical features of the original dataset only, we get the reduced version with only 2 features, named as `dim_1` and `dim_2`. We save the reduced dataset in a pandas dataframe called `umap_components`. We specify the index of this dataframe to be the same as the index of the original dataframe (`index=dataset.index`). Then, we concatenate the reduced dataset with the `category` column of the original dataset and save it in a new dataframe (`data_after_umap`). We use this dataframe to visualize the results by calling the `visualize_data()` function.


In [10]:
def run_umap(min_dist:float, n_neighbors:int, dataset:pd.DataFrame, numerical_cols:list):
    '''
    This method is used to create the UMAP object that will perform the
    dimensionality reduction using the fit_transform method. The results of
    dimensionality reduction are saved in a pandas dataframe that contain the 2
    columns holding the new, reduced dimensions as well as a third column for
    the category of the sample. 

    Args:
     - min_dist: UMAP hyperparameter
     - n_neighbors: UMAP hyperparameter
     - dataset: a pandas dataframe containing the data
     - numerical_cols: a list of the columns containing numerical data

    Returns: nothing
    '''  
    
    umap = UMAP(n_components=2, n_neighbors=n_neighbors, 
                min_dist=min_dist, random_state=0)
    
    umap_components = pd.DataFrame(
                        data=umap.fit_transform(dataset[numerical_cols]), 
                        columns=['dim_1', 'dim_2'],
                        index=dataset.index #this is important for the concatenation in the next command
                        )

    data_after_umap = pd.concat([umap_components, dataset['category']], axis=1)
    data_after_umap.columns = ['dim_1', 'dim_2', 'category']

    visualize_data(data_after_umap, 
                   f'UMAP embeddings - n_neighbors={n_neighbors} and min_dist={min_dist}')

In [11]:
n_neighbors_slider = widgets.IntSlider(value=30, min=10, max=100, step=5, description='n_neighbors:')
min_dist_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.05, description='min_dist:')
widgets.interact_manual(run_umap, min_dist=min_dist_slider, n_neighbors=n_neighbors_slider, \
                        dataset = widgets.fixed(dataset), numerical_cols = widgets.fixed(numerical_cols));

interactive(children=(FloatSlider(value=0.5, description='min_dist:', max=1.0, step=0.05), IntSlider(value=30,…

In [12]:
# You can use this cell to experiment more with the hyperparameters and compare 
# the plots. You will need to uncomment the below lines.

# n_neighbors_slider = widgets.IntSlider(value=30, min=10, max=100, step=5, description='n_neighbors:')
# min_dist_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.05, description='min_dist:')
# widgets.interact_manual(run_umap, min_dist=min_dist_slider, n_neighbors=n_neighbors_slider, dataset = widgets.fixed(dataset));

💡 **CHECKPOINT:**
- Experiment with different values of `n_neighbors` and `min_dist`. What do you notice? How do different values of these hyperparameters affect the grouping of different categories? At what values do the samples of each category cluster together? 

<a id='conclusion'></a>
## 3. Conclusion


<div align="center">

|                                 | PCA                     | tSNE                           | UMAP                                      |
|---------------------------------|-------------------------|--------------------------------|-------------------------------------------|
| **objective**                   | maximize variance       | preserve local structure       | preserve local and global structure       |
| **linearity**                   | linear                  | non-linear                     | non-linear                                |
| **scalability**                 | fast for large datasets | the slowest for large datasets | slow for large datasets                   |
| **most important parameters**   | `n_components`          | `n_components`, `perplexity`   | `n_components`, `n_neighbors`, `min_dist` |
| **support for new data points** | Yes                     | Not directly                   | Not directly                              |

</div>

In this tutorial we have looked at the unsupervised part of machine learning. First, we looked at what unsupervised learning is. Then we explained what dimensionality reduction is. We focused on 3 different dimensionality reduction techniques: PCA, t-SNE and UMAP. The table above summarizes the main features of each of these three dimensionality reduction techniques.  
As we have seen, PCA focuses on maximising the variance preserved after reducing the dimensions of the dataset, while t-SNE and UMAP focus on preserving the local and global structure. PCA is a linear technique, whereas t-SNE and UMAP are non-linear. When applied to large datasets, PCA is the fastest and t-SNE is the slowest. In terms of tuned hyperparameters, in all three methods we used the `n_components`, which determines the number of features in the lower dimensional space. t-SNE has an additional parameter, `perplexity`, which controls the trade-off between preserving local and global structure, while in UMAP this is done by two hyperparameters: `min_dist` and `n_neighbours`. Finally, only PCA can be applied to new points (e.g. computed on the training set and applied to samples of the test set), while for t-SNE and UMAP the models have to be recalculated whenever new samples are available.

**References:**

- "Machine Learning with Pytorch and Scikit-Learn" - Sebastian Raschka, Yuxi Liu, Vahid Mirjalili, Dmytro Dzhulgakov.
- ["tSNE clearly explained"](https://towardsdatascience.com/t-sne-clearly-explained-d84c537f53a) - Kemal Erdem
- ["Understanding UMAP"](https://pair-code.github.io/understanding-umap/) - Andy Coenen, Adam Pearce